# Этап 2: Обучение и Оценка Модели Обнаружения Объектов

На этом этапе проекта мы переходим к обучению нейронной сети для обнаружения сноубордистов на подготовленном датасете. 

**Цель:** получить высокопроизводительную и точную модель-детектор, способную эффективно локализовать объекты класса 'snowboarder' на изображениях и видео.

**Примечание для читателя:**
Данный ноутбук демонстрирует процесс обучения и оценки моделей. Все конфигурационные параметры и логирование экспериментов могут быть дополнительно настроены и отслежены с помощью инструментов, таких как Weights & Biases, для более глубокого анализа.

---

**Ключевые особенности этого этапа:**

* **Централизованное управление параметрами:** Все ключевые конфигурационные параметры (такие как тип модели, количество эпох, размер изображения, использование W&B и другие) определяются в одном словаре `training_config`, что упрощает модификацию и контроль над экспериментами.
* **Интеграция Weights & Biases (W&B):** Весь процесс обучения и оценки модели полностью интегрирован с W&B. Это позволяет в реальном времени отслеживать метрики (потери, точность, полнота), визуализировать графики (PR-кривые, матрицы ошибок), фиксировать все параметры запуска и артефакты (обученные модели), а также сравнивать результаты различных экспериментов в едином облачном интерфейсе.
* **Динамическое именование запусков:** Локальные директории для сохранения результатов обучения и тестирования, а также соответствующие раны в W&B, автоматически получают уникальные, последовательные имена. Это предотвращает перезапись данных и упрощает навигацию по множеству экспериментов.
* **Упорядоченная структура выходных данных:** Все сгенерированные артефакты (веса моделей, графики обучения, логи, результаты валидации) сохраняются в четко определенной иерархии в папке `runs/` (например, `runs/detect/`, `runs/wandb/`), обеспечивая чистоту проекта и легкий доступ к результатам.

В первой части мы сфокусируемся на обучении **YOLOv8** как основной архитектуры, благодаря её балансу между скоростью и точностью, что критически важно для дальнейшей интеграции в систему слежения.


In [ ]:
# Импорт необходимых системных и стандартных библиотек
import os
import sys
import time
import wandb

# Добавление корневой директории проекта в sys.path
# Это позволяет импортировать вспомогательные модули из папок, расположенных на одном уровне с 'notebooks/' (например, из 'scripts/')
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

# Импорт основной библиотеки для работы с моделью и отображением
from ultralytics import YOLO

# Импорт пользовательских вспомогательных утилит
from scripts.visualization_utils import display_and_log_multiple_image_artifacts # Для визуализации и логирования изображений
from scripts.utils import get_next_run_name, check_yolo_dataset_paths # Утилиты для генерации имен запусков и проверки путей датасета

## 1. Обучение YOLOv8

Будем использовать фреймворк Ultralytics YOLOv8 для обучения модели.

### 1.1. Конфигурация Датасета

Перед началом обучения необходимо убедиться, что файл конфигурации `dataset.yaml` корректно указывает на подготовленные данные.

In [ ]:
# Путь к файлу dataset.yaml
dataset_yaml_path = '../resources/dataset.yaml'

# Проверяем пути в dataset.yaml
print("Проверка файла dataset.yaml и доступности указанных в нем путей:")
paths_are_ok = check_yolo_dataset_paths(dataset_yaml_path)

### 1.2. Загрузка предобученной модели и обучение

Мы начнем с загрузки предобученной модели YOLOv8 (`yolov8n.pt` — 'n' означает 'nano', самую маленькую и быструю версию). Использование предобученной модели на большом датасете (например, COCO) позволяет нам воспользоваться уже выученными признаками и значительно ускорить процесс обучения на наших специфических данных (transfer learning).

**Параметры обучения:**
* **`data`:** Путь к файлу `dataset.yaml`.
* **`epochs`:** Количество эпох обучения. Для начала, 50-100 эпох будет достаточно для демонстрации. Если модель будет недообучена, можно увеличить.
* **`imgsz`:** Размер изображения, до которого оно будет масштабироваться перед подачей в модель. 640 — стандартное значение.
* **`batch`:** Размер батча. Рекомендуется подбирать в зависимости от доступной GPU памяти. `-1` для автоматического подбора.
* **`name`:** Название эксперимента. Результаты обучения будут сохранены в `runs/detect/<name>/`.

In [ ]:
# --- Загрузка предобученной модели и обучение ---

# Конфигурация параметров обучения и W&B
training_config = {
    "model_name_prefix": 'yolo11n',              # Префикс для имени модели
    "epochs": 100,                               # Количество эпох
    "imgsz": 640,                                # Размер изображения
    "batch_size": -1,                            # Размер батча (-1 для автоподбора)
    "patience": 50,                              # Количество эпох без улучшения на val loss
    "resume": False,                             # Продолжить с контрольной точки
    "project_name": "snowboard-tracking",        # Имя проекта в Weights & Biases
    "run_base_prefix": "snowboarder_detection",  # Базовый префикс для нумерации запусков
    "use_wandb": True,                           # Использовать Weights & Biases для логирования
    "device": None                               # Или 0, 'cpu'
}

print("Загрузка предобученной модели и обучение")

# Определяем имя для текущего тренировочного запуска
current_train_run_name = get_next_run_name(
    f"{training_config['model_name_prefix']}_{training_config['run_base_prefix']}"
)

# Инициализация W&B run (опционально)
if training_config['use_wandb']:
    print(f"Инициализация Weights & Biases для запуска '{current_train_run_name}'...")
    wandb.init(
        project=training_config['project_name'],
        name=current_train_run_name,
        config=training_config, # Логируем все параметры конфигурации
        dir='../runs'
    )
else:
    print("Логирование в Weights & Biases отключено.")
    os.environ["WANDB_MODE"] = "dryrun" # Это заставит W&B не логировать, но не вызовет ошибок, если wandb.init() не был вызван.

# Загрузка модели
model = YOLO(f'{training_config["model_name_prefix"]}.pt') 
print(f"Модель YOLO11 ({training_config['model_name_prefix']}.pt) загружена.")

# Обучение модели
print(f"\nНачинаем обучение модели YOLO11. Результаты будут сохранены в {os.path.join('runs', 'detect', current_train_run_name)}/")

start_time = time.time()
results = model.train(
    data=dataset_yaml_path,               # Путь к файлу конфигурации датасета
    epochs=training_config['epochs'],     # Количество эпох
    imgsz=training_config['imgsz'],       # Размер изображения
    batch=training_config['batch_size'],  # Размер батча (-1 для автоподбора)
    name=current_train_run_name,          # Название эксперимента (для папки runs/detect/)
    project='../runs/detect',             # Подпапка внутри 'runs/' (т.е. runs/detect/...)
    device=training_config['device'],     # Убедитесь, что GPU доступен, иначе оставьте пустым для CPU или 'cpu'
    patience=training_config['patience'], # Количество эпох без улучшения на val loss
    resume=training_config['resume'],     # Продолжить обучение с последней контрольной точки
    val=True,                             # Проводить валидацию на каждой эпохе
    # callbacks=True                      # Необязательно, если wandb.init() был вызван, YOLOv8 обычно сам подключит W&B колбэки.
)
end_time = time.time()

training_duration_seconds = end_time - start_time
training_duration_minutes = training_duration_seconds / 60

print("\nОбучение YOLO11 завершено.")

# Сохраняем имя папки с обучением для дальнейшего использования (например, для тестирования)
latest_train_run_dir = model.trainer.save_dir
print(f"Результаты обучения сохранены в: {latest_train_run_dir}")

### 1.3. Анализ результатов обучения YOLOv8

Обучение модели YOLOv8n завершилось успешно, показав отличные метрики на валидационном наборе.

**Ключевые показатели после 100 эпох:**

* **Итоговая оценка на валидационном наборе (по модели `best.pt`):**
    * `Precision (P)`: **0.969** - Из всех предсказаний, почти 97% были корректными.
    * `Recall (R)`: **1** - Модель обнаружила абсолютно всех (или почти всех) сноубордистов на валидационных изображениях.
    * `mAP50`: **0.992** - Практически идеальная средняя точность при пороге IoU 0.5.
    * `mAP50-95`: **0.868** - Очень высокий показатель средней точности по диапазону порогов IoU, свидетельствующий о превосходном качестве предсказанных ограничивающих рамок.

* **Скорость инференса на GPU (NVIDIA GeForce RTX 2060)**:
    * **1.1** мс на инференс + 1.5 мс на постобработку = 2.6 мс общее время на изображение
    * Это составляет примерно ~**385** кадров в секунду (FPS) для полного пайплайна
    * Чистый инференс: ~**909** FPS (только нейросеть без постобработки)


* **Характеристики модели:**
    * **3,005,843** параметра - компактная архитектура
    * **8.1** GFLOPs - вычислительная сложность
    * **6.2** MB размер модели - идеально для мобильных устройств

Такие результаты свидетельствуют о высоком качестве подготовленного датасета и эффективности выбранной архитектуры YOLOv8 для данной задачи. Модель успешно выучила признаки сноубордистов и демонстрирует отличную способность к их обнаружению.

### 1.4. Визуализация результатов обучения YOLOv8

Ultralytics автоматически генерирует полезные графики во время обучения, которые позволяют отслеживать прогресс модели и выявлять потенциальные проблемы (например, переобучение или недообучение). Эти графики находятся в папке, где были сохранены результаты обучения (`yolov8n_snowboarder_v1`).

Мы отобразим ключевые графики:
* `results.png`: Сводный график потерь и метрик (mAP, Precision, Recall) по эпохам.
* `PR_curve.png`: Precision-Recall кривая.
* `confusion_matrix.png`: Матрица ошибок.

In [ ]:
# Путь к папке с результатами обучения
train_results_dir = model.trainer.save_dir

print(f"\n 1.4 Визуализация результатов обучения")
print(f"Графики обучения сохранены в: {train_results_dir}")

# Файлы графиков, которые Ultralytics генерирует в директории обучения
training_plot_files = [
    'results.png',
    'BoxPR_curve.png',
    'confusion_matrix.png'
]

# Словарь для передачи индивидуальной ширины, если нужно
training_plot_widths = {
    'BoxPR_curve.png': 600,
    'confusion_matrix.png': 600
}

# Используем display_and_log_multiple_image_artifacts только для отображения в ноутбуке
# W&B уже логирует эти файлы автоматически.
display_and_log_multiple_image_artifacts(
    base_dir=train_results_dir,
    image_filenames=training_plot_files,
    prefix_title="График обучения",
    wandb_artifacts_dict=None,
    wandb_key_prefix=None,
    widths=training_plot_widths
)

## 2. Тестирование модели на независимой тестовой выборке

После завершения обучения и получения финальных весов модели, крайне важно провести её объективную оценку на **независимом тестовом наборе данных**. Этот набор состоит из изображений, которые модель не видела ни на этапе обучения, ни на этапе валидации. Такая оценка дает наиболее точное представление об обобщающей способности модели и её производительности в реальных условиях.

Мы используем веса `best.pt`, полученные в результате обучения, для проведения валидации на тестовом наборе.

In [ ]:
print(f"2. Оценка Модели на Независимом Тестовом Наборе")

# Загрузка лучшей обученной модели
best_model_path = os.path.join(latest_train_run_dir, 'weights', 'best.pt')

print(f"Загружаем лучшую модель из: {best_model_path}")
loaded_model = YOLO(best_model_path)

# Определяем имя для текущего тестового запуска
test_run_base_name = f"test_{training_config['model_name_prefix']}_{training_config['run_base_prefix']}"
current_test_run_name = get_next_run_name(test_run_base_name)

# Проведение валидации на тестовом наборе
print("\nЗапускаем валидацию на тестовом наборе...")
test_results = loaded_model.val(
    data=dataset_yaml_path,          # Используем тот же dataset.yaml
    split='test',                    # Указываем, что валидацию нужно провести на тестовом наборе
    imgsz=training_config['imgsz'],  # Используем imgsz из конфига обучения
    conf=0.25,                       # Порог уверенности для детекции (можно настроить)
    iou=0.7,                         # Порог IoU для NMS (можно настроить)
    plots=True,                      # Генерировать графики результатов
    name=current_test_run_name,      # Динамическое имя для тестового запуска
    project='../runs/detect'         # Подпапка внутри 'runs/' (т.е. runs/detect/...)
)

print("\nВалидация на тестовом наборе завершена.")

# Вывод основных метрик на тестовом наборе
print("\nМетрики на тестовом наборе:")
final_p = test_results.box.p.item() if test_results.box.p is not None else float('nan')
final_r = test_results.box.r.item() if test_results.box.r is not None else float('nan')
final_map50 = test_results.box.map50.item() if test_results.box.map50 is not None else float('nan')
final_map = test_results.box.map.item() if test_results.box.map is not None else float('nan')

print(f"   Precision (P): {final_p:.3f}")
print(f"   Recall (R): {final_r:.3f}")
print(f"   mAP50: {final_map50:.3f}")
print(f"   mAP50-95: {final_map:.3f}")

# Логирование финальных метрик тестового набора в W&B (если W&B был активен)
if training_config['use_wandb']:
    wandb.log({
        "test/precision": final_p,
        "test/recall": final_r,
        "test/mAP50": final_map50,
        "test/mAP50-95": final_map
    })
    # Логирование времени обучения
    if 'training_duration_seconds' in locals():
        wandb.log({
            "training_duration_seconds": training_duration_seconds,
            "training_duration_minutes": training_duration_minutes
        })

# Путь к сгенерированным графикам из теста
plots_test_dir = test_results.save_dir

print(f"\nПримеры предсказаний и графики оценки на тестовом наборе сохранены в: {plots_test_dir}")

# Файлы графиков для тестового набора
test_plot_files = [
    'val_batch0_pred.jpg', # Пример предсказаний на первом батче валидации
    'BoxPR_curve.png',     # PR-кривая для теста
    'confusion_matrix.png' # Матрица ошибок для теста, если нужна
]

display_and_log_multiple_image_artifacts(
    base_dir=plots_test_dir,
    image_filenames=test_plot_files,
    prefix_title="Тестовый результат",
    wandb_artifacts_dict=None, # Не передаем словарь для W&B артефактов
    wandb_key_prefix=None
)

# Завершаем W&B run в самом конце эксперимента
if training_config['use_wandb']:
    wandb.finish()

### 2.2. Анализ Результатов на Тестовой Выборке

Результаты валидации на независимой тестовой выборке подтверждают высокую обобщающую способность обученной модели YOLOv8n.

**Ключевые показатели на тестовом наборе (63 изображения):**

* `Precision (P)`: **0.984** - Модель демонстрирует крайне низкое количество ложных срабатываний.
* `Recall (R)`: **1.000** - Модель успешно обнаружила всех сноубордистов, присутствующих в тестовом наборе. Это критически важно для задачи трекинга, так как пропущенные детекции значительно ухудшают качество трекинга.
* `mAP50`: **0.994** - Практически идеальная средняя точность при пороге IoU 0.5, что говорит о высочайшем качестве локализации объектов.
* `mAP50-95`: **0.842** - Очень сильный показатель, указывающий на хорошую точность рамок даже при строгих порогах IoU.

Эти метрики являются **отличным фундаментом** для построения надежной системы слежения за сноубордистом. Модель-детектор готова к интеграции с алгоритмами трекинга.

## **3 Завершение этапа и выводы**

Обучение и тщательная оценка модели YOLOv8 для обнаружения сноубордистов успешно завершены. Модель продемонстрировала высокую производительность как на валидационной, так и на полностью независимой тестовой выборке, подтверждая свою способность эффективно обобщать и детектировать объекты на ранее невиданных данных.

**Ключевые показатели производительности обученной модели YOLOv8n:**

**На независимом тестовом наборе (63 изображения, 41 экземпляр сноубордиста):**

* `Precision (P)`: **0.984** - Модель демонстрирует крайне низкое количество ложных срабатываний, что означает высокую надежность детекций.
* `Recall (R)`: **1.000** - Модель успешно обнаружила **всех** сноубордистов, присутствующих в тестовом наборе. Это критически важно для задачи трекинга, так как пропущенные детекции значительно ухудшают качество отслеживания.
* `mAP50`: **0.995** - Практически идеальная средняя точность при пороге Intersection Over Union (IoU) 0.5. Это говорит о высочайшем качестве как детекции, так и локализации ограничивающих рамок.
* `mAP50-95`: **0.842** - Очень сильный показатель, указывающий на хорошую точность рамок даже при более строгих порогах IoU (от 0.5 до 0.95).

**Характеристики и скорость инференса обученной модели (на NVIDIA GeForce RTX 2060):**

* **Скорость инференса:**
    * Чистый инференс (только нейросеть): около **1.1 мс** на изображение.
    * Полный пайплайн (инференс + постобработка): около **2.6 мс** на изображение (1.1 мс инференс + 1.5 мс постобработка).
    * Это соответствует примерно **~385 кадрам в секунду (FPS)** для полного пайплайна, что идеально подходит для задач отслеживания в реальном времени.
    * _Примечание: Во время тестовой валидации скорость инференса могла быть снижена за счет особенностей обработки тестового батча и дополнительного логирования. При работе модели в реальном приложении ожидается стабильная скорость на уровне 1-2 мс на изображение._

* **Архитектурные особенности модели YOLOv8n:**
    * **3,005,843** параметра - указывает на компактную и легковесную архитектуру, что снижает требования к памяти и вычислительным ресурсам.
    * **8.1 GFLOPs** - вычислительная сложность модели. Относительно низкое значение подтверждает ее эффективность.
    * **6.2 MB** размер файла модели - чрезвычайно малый размер, что делает ее идеальной для развертывания на устройствах с ограниченными ресурсами, включая мобильные платформы и встраиваемые системы.

Все ключевые метрики обучения и оценки, а также параметры эксперимента и визуализации, были автоматически залогированы в Weights & Biases. Это обеспечивает полную прозрачность и воспроизводимость каждого запуска, позволяя легко сравнивать результаты и отслеживать прогресс проекта. Локальные артефакты (обученные веса модели, графики обучения и валидации) аккуратно сохранены в структурированной директории `runs/detect/`.

Эта высокоэффективная и тщательно протестированная модель обнаружения объектов теперь является прочной основой для следующего этапа проекта — реализации системы отслеживания и центрирования сноубордиста в видеопотоке.

### Сравнительный анализ производительности моделей YOLOv8n, YOLOv8s и YOLO11n

Для принятия обоснованного решения о выборе оптимальной модели для задачи отслеживания в реальном времени, был проведен сравнительный анализ трех версий YOLO: `yolov8n` (nano), `yolov8s` (small) и `yolo11n` (nano).

| Метрика / Характеристика | YOLOv8n (Nano) | YOLOv8s (Small) | YOLO11n (Nano) | Комментарий                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| :---------------------- | :------------- | :-------------- | :------------- | :---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **Параметры (общие)** | 3.0M           | 11.1M           | **2.6M** | `yolo11n` является самой компактной моделью с наименьшим количеством параметров.                                                                                                                                                                                                                                                                                                                                                                                                     |
| **GFLOPs (общие)** | 8.1 GFLOPs     | 28.4 GFLOPs     | **6.3 GFLOPs** | `yolo11n` является самой легкой с точки зрения вычислительной сложности, что способствует высокой скорости инференса.                                                                                                                                                                                                                                                                                                                                                         |
| **Время обучения (100 эпох)** | ~5 минут      | ~7 минут (0.114 часа) | **~4.26 минут (0.071 часа)** | `yolo11n` обучается быстрее всех, демонстрируя высокую эффективность.                                                                                                                                                                                                                                                                                                                                       |
| **mAP50 (Тест)** | **0.995** | 0.994           | 0.994          | Все модели демонстрируют исключительно высокие и практически идентичные mAP50, что говорит о высоком качестве детекции при стандартном пороге IoU.                                                                                                                                                                                                                                                                                                    |
| **mAP50-95 (Тест)** | 0.842          | 0.834           | **0.859** | `yolo11n` достигает наивысшего `mAP50-95`, что указывает на лучшую точность локализации ограничивающих рамок даже при строгих порогах IoU. Это критически важный показатель качества модели.                                                                                                                                                            |
| **Precision (P) (Тест)** | **0.984** | 0.982           | 0.974          | Все модели имеют очень высокую точность (минимум ложных срабатываний). `yolo11n` лишь незначительно уступает.                                                                                                                                                                                                                                                                                                                                                                                   |
| **Recall (R) (Тест)** | **1.000** | 0.984           | **1.000** | `yolo11n` и `yolov8n` достигают идеального Recall, не пропуская ни одного объекта в тестовом наборе, что является ключевым требованием для надежного трекинга.                                                                                                                                                                                                                                                                                              |
| **Скорость инференса (чистый / полный пайплайн, мс/изображение)** | ~1.1 / ~2.6 | ~8.4 / ~11.9 | **~1.1 / ~2.4** | **`yolo11n` показывает выдающуюся скорость инференса, сравнимую с `yolov8n`, и значительно превосходит более крупную `yolov8s`.** |
| **FPS (полный пайплайн, оценка)** | ~385 FPS       | ~84 FPS         | **~416 FPS** | `yolo11n` является самой быстрой моделью в полном пайплайне, что делает ее наиболее предпочтительной для приложений реального времени.                                                                                                                                                                                                                                                                                                                              |

### Окончательные выводы по выбору модели и дальнейшим экспериментам:

На основании проведенного всестороннего сравнительного анализа трех моделей YOLO, можно сделать следующие ключевые выводы:

1.  **YOLO11n - безоговорочный лидер для данной задачи:** Модель `yolo11n` демонстрирует **наилучший компромисс между высокой точностью и исключительной производительностью**. Она не только превосходит `yolov8n` и `yolov8s` по ключевым метрикам точности (особенно `mAP50-95`), но при этом является самой компактной, обучается быстрее всех и обеспечивает наивысший FPS в реальном времени. Ее способность обнаруживать все объекты (`Recall = 1.000`) при отличной точности локализации делает её идеальным выбором для надежной системы отслеживания сноубордистов.

2.  **Подтверждение избыточности крупных моделей:** Результаты экспериментов с `yolov8s` ясно показывают, что увеличение сложности модели (от `n` к `s`) не приносит существенного прироста точности на нашем специфическом датасете, а ведет к значительному увеличению вычислительных затрат и снижению скорости. Это подтверждает, что для данной задачи и размера данных, более крупные версии YOLO (например, `yolov8m`, `yolov8l`, `yolov8x`) **не являются целесообразными и лишь приведут к неэффективному использованию ресурсов**.

### Стоит ли пробовать YOLO12n?

Ваш вопрос о `yolo12n` закономерен. Учитывая, как хорошо `yolo11n` показала себя, есть несколько соображений:

* **Потенциал:** Каждая новая версия YOLO стремится к улучшению. `yolo12n` может предложить дальнейшие оптимизации архитектуры или тренировки, которые теоретически могли бы привести к небольшому, но заметному приросту точности при сохранении высокой скорости.
* **Закон убывающей отдачи:** С другой стороны, мы уже достигли очень высоких метрик. `mAP50-95 = 0.859` и `Recall = 1.000` — это выдающиеся результаты. Прирост от `yolo12n`, если он и будет, скорее всего, окажется минимальным (доли процента), а время, затраченное на обучение и оценку новой модели, может не окупиться этим незначительным улучшением.
* **Фокус проекта:** Если текущая цель — создать рабочую систему отслеживания, `yolo11n` уже более чем готова к интеграции. Время, которое вы потратите на дальнейшие бенчмаркинг моделей, можно было бы инвестировать в реализацию и оптимизацию самого алгоритма отслеживания.

**Рекомендация:** На данном этапе, если `yolo11n` полностью удовлетворяет вашим требованиям по точности и скорости, **нет острой необходимости немедленно переходить к `yolo12n`**. Если в процессе интеграции в систему отслеживания или при работе с более сложными сценариями обнаружатся недостатки `yolo11n`, тогда можно вернуться к идее попробовать `yolo12n` или другие дальнейшие оптимизации (например, увеличение `imgsz` для `yolo11n` или дополнительное дообучение).

**Итог:** Вы получили исключительную модель-детектор в лице `yolo11n`. Это отличный результат для перехода к следующей фазе проекта!